In [17]:
# load all the toolboxes required
import pandas as pd
import matplotlib as plt
import numpy as np
#from patsy import dmatrix
import requests, zipfile, io
from tqdm import tqdm
from sklearn.preprocessing import scale
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import linear_model

In [18]:
# load the data directly from the website
dfs = []

for year in tqdm(range(2013, 2019)):
    r = requests.get('http://www.tennis-data.co.uk/{0}/{0}.zip'.format(year))
    z = zipfile.ZipFile(io.BytesIO(r.content))
    df = pd.read_excel(z.open('{0}.xlsx'.format(year)))
    df['year'] = year
    dfs.append(df)

df = pd.concat(dfs)
print(df)


100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:08<00:00,  1.42s/it]

      ATP  Location              Tournament       Date       Series    Court  \
0       1  Brisbane  Brisbane International 2012-12-31       ATP250  Outdoor   
1       1  Brisbane  Brisbane International 2012-12-31       ATP250  Outdoor   
2       1  Brisbane  Brisbane International 2012-12-31       ATP250  Outdoor   
3       1  Brisbane  Brisbane International 2012-12-31       ATP250  Outdoor   
4       1  Brisbane  Brisbane International 2013-01-01       ATP250  Outdoor   
...   ...       ...                     ...        ...          ...      ...   
2632   67    London             Masters Cup 2018-11-16  Masters Cup   Indoor   
2633   67    London             Masters Cup 2018-11-16  Masters Cup   Indoor   
2634   67    London             Masters Cup 2018-11-17  Masters Cup   Indoor   
2635   67    London             Masters Cup 2018-11-17  Masters Cup   Indoor   
2636   67    London             Masters Cup 2018-11-18  Masters Cup   Indoor   

     Surface        Round  Best of     

In [19]:
# Create new data object for the real classification
col_names =  ['classifier_labels', 'World_rank', 'World_rank_points']
index_len=len(df['ATP'])*2
index=range(1, index_len)
classifier_input  = pd.DataFrame(index=range(0, index_len), columns = col_names)
unique_person=np.concatenate((df['Winner'], df['Loser']), 0)
surfaces=np.concatenate((df['Surface'], df['Surface']), 0)


In [20]:
#clean data up
df['W3'].fillna(0, inplace=True)
df['L3'].fillna(0, inplace=True)


In [21]:
# Create labels to train the calssifier on (winner=1 & loser =0)
zero_data=np.zeros((len(df['ATP'])))
ones_data=np.ones((len(df['ATP'])))
classifier_labels=np.concatenate((ones_data, zero_data), 0)

Set_diff1=df['Wsets']-df['Lsets']
Set_diff2=df['Lsets']-df['Wsets']
classifier_graded_input=np.concatenate((Set_diff1, Set_diff2), 0)

Win_point=df['W1']+df['W2']
Loose_point=df['L1']+df['L2']
Point_diff1=Win_point-Loose_point
Point_diff2=Loose_point-Win_point
classifier_graded_input2=np.concatenate((Point_diff1, Point_diff2), 0)
print(classifier_graded_input2)

[ 4. -1.  6. ... -3. -8. -5.]


In [22]:
# Create predictors that match the dimensions of the classifier labels
# THIS IS THE PLACE WHERE I NEED TO INTRODUCE NEW PREDICTORS
World_rank_diff1=df['WRank']-df['LRank']
World_rank_diff2=df['LRank']-df['WRank']
World_point_diff1=df['WPts']-df['LPts']
World_point_diff2=df['LPts']-df['WPts']

World_rank=np.concatenate((df['WRank'],df['LRank']), 0)
World_rank_points=np.concatenate((df['WPts'],df['LPts']),0)
World_rank_diff=np.concatenate((World_rank_diff1,World_rank_diff2),0)
World_point_diff=np.concatenate((World_point_diff1,World_point_diff2),0)

In [23]:
# Clean up the data to exclude NaNs
classifier_input = pd.DataFrame(dict(classifier_labels=classifier_labels,classifier_graded_input=classifier_graded_input,
                                     classifier_graded_input2=classifier_graded_input2, World_rank=World_rank, World_rank_point=World_rank_points, 
                                     World_rank_diff=World_rank_diff,World_point_diff=World_point_diff,Player=unique_person, surfaces=surfaces))

classifier_input=classifier_input.dropna(subset=['classifier_labels','classifier_graded_input','classifier_graded_input2', 'World_rank', 'World_rank_point','World_rank_diff', 'World_point_diff', 'Player', 'surfaces'])
Players_un=pd.DataFrame(dict(name=classifier_input['Player']))
Surface=pd.DataFrame(dict(name=classifier_input['surfaces']))

print(Players_un)

               name
0          Mayer F.
1       Nieminen J.
2      Nishikori K.
3      Baghdatis M.
4        Istomin D.
...             ...
31509      Isner J.
31510      Cilic M.
31511    Federer R.
31512   Anderson K.
31513   Djokovic N.

[30948 rows x 1 columns]


In [24]:
player_in_data=Players_un.name.unique()
for unique in range(0, len(player_in_data)):
    Players_un['Player{0}'.format(unique)]=np.zeros((len(classifier_input['classifier_labels'])))
    Players_un['Player{0}'.format(unique)][classifier_input['Player']==player_in_data[unique]]=1
print(Players_un)
surface_in_data=Surface.name.unique()
for sur in range(0, len(surface_in_data)):
    Surface['Surface{0}'.format(sur)]=np.zeros((len(classifier_input['classifier_labels'])))
    Surface['Surface{0}'.format(sur)][classifier_input['surfaces']==surface_in_data[unique]]=1



C:\Users\mrowl\Miniconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


               name  Player0  Player1  Player2  Player3  Player4  Player5  \
0          Mayer F.      1.0      0.0      0.0      0.0      0.0      0.0   
1       Nieminen J.      0.0      1.0      0.0      0.0      0.0      0.0   
2      Nishikori K.      0.0      0.0      1.0      0.0      0.0      0.0   
3      Baghdatis M.      0.0      0.0      0.0      1.0      0.0      0.0   
4        Istomin D.      0.0      0.0      0.0      0.0      1.0      0.0   
...             ...      ...      ...      ...      ...      ...      ...   
31509      Isner J.      0.0      0.0      0.0      0.0      0.0      0.0   
31510      Cilic M.      0.0      0.0      0.0      0.0      0.0      0.0   
31511    Federer R.      0.0      0.0      0.0      0.0      0.0      0.0   
31512   Anderson K.      0.0      0.0      0.0      0.0      0.0      0.0   
31513   Djokovic N.      0.0      0.0      0.0      0.0      0.0      0.0   

       Player6  Player7  Player8  ...  Player581  Player582  Player583  \
0

IndexError: index 590 is out of bounds for axis 0 with size 3

In [ ]:
# Set up the classifier_labels and divide the data into train and test set
clf = LogisticRegression()
X = classifier_input[['World_rank', 'World_rank_point', 'World_rank_diff', 'World_point_diff']]
for unique in tqdm(range(0, len(player_in_data))):
    X['Player{0}'.format(unique)]=Players_un['Player{0}'.format(unique)]
print(X)    
X = scale(X)
Y=classifier_input['classifier_labels']
Y_continuous=classifier_input['classifier_graded_input']
Y_continuous2=classifier_input['classifier_graded_input2']

#X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=0)

In [25]:
# test a linear svm 
vector_machine = svm.SVC(kernel='linear')
for fold in tqdm(range(1, 5)):
    X_train, X_test, y_train, y_test = train_test_split(X, Y)
    svm_model = vector_machine.fit(X_train, y_train)
    print(svm_model.score(X_test, y_test))


 25%|████████████████████▊                                                              | 1/4 [07:22<22:06, 442.05s/it]

0.6498642884839085



 50%|█████████████████████████████████████████▌                                         | 2/4 [14:44<14:44, 442.25s/it]

0.6485717978544655



 75%|██████████████████████████████████████████████████████████████▎                    | 3/4 [22:12<07:23, 443.82s/it]

0.6520615225539614



100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [29:46<00:00, 446.56s/it]

0.6507690319245185


In [ ]:
# test a linear sv-regression
vector_machine_regression = svm.SVR(kernel='linear')

for fold in tqdm(range(1, 5)):
    X_train, X_test, y_train, y_test = train_test_split(X, Y_continuous)
    svm_model_reg = vector_machine_regression.fit(X_train, y_train)
    print(svm_model_reg.predict(X_test))


In [92]:
# combine a regression svr and feed it into a svc
vector_machine_regression = svm.SVR(kernel='linear')
vector_machine = svm.SVC(kernel='rbf')

for fold in tqdm(range(1, 5)):
    X_train, X_test, y_train_cont, y_test_cont = train_test_split(X, Y_continuous, random_state=fold)
    X_train, X_test, y_train_cont2, y_test_cont2 = train_test_split(X, Y_continuous2, random_state=fold)

    X_train, X_test, y_train_clas, y_test_clas = train_test_split(X, Y, random_state=fold)

    svm_model_reg = vector_machine_regression.fit(X_train, y_train_cont)
    continuous_input_svc_train=svm_model_reg.predict(X_train)
    continuous_input_svc_test=svm_model_reg.predict(X_test)
    
    svm_model_reg2 = vector_machine_regression.fit(X_train, y_train_cont2)
    continuous_input_svc_train2=svm_model_reg.predict(X_train)
    continuous_input_svc_test2=svm_model_reg.predict(X_test)
    
    continuous_input_train = pd.DataFrame(dict(continuous_input_svc_train=continuous_input_svc_train, continuous_input_svc_train2=continuous_input_svc_train2))
    continuous_input_test = pd.DataFrame(dict(continuous_input_svc_test=continuous_input_svc_test, continuous_input_svc_test2=continuous_input_svc_test2))

    
    X_comb_train = continuous_input_train[['continuous_input_svc_train', 'continuous_input_svc_train2']]
    X_comb_train = scale(X_comb_train)
    
    X_comb_test = continuous_input_test[['continuous_input_svc_test', 'continuous_input_svc_test2']]
    X_comb_test = scale(X_comb_test)
    
    svm_model = vector_machine.fit(X_comb_train, y_train_clas)
    print(svm_model.score(X_comb_test, y_test_clas))




















  0%|                                                                                            | 0/4 [00:00<?, ?it/s]

0.653612511309293





















 25%|████████████████████▊                                                              | 1/4 [01:46<05:19, 106.57s/it]

0.663693938218948





















 50%|█████████████████████████████████████████▌                                         | 2/4 [03:35<03:34, 107.30s/it]

0.6586532247641205





















 75%|██████████████████████████████████████████████████████████████▎                    | 3/4 [05:21<01:46, 106.84s/it]

0.663693938218948





















100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [07:10<00:00, 107.50s/it]


















